<a href="https://colab.research.google.com/github/arbakaydemir/PySparkCommands/blob/main/Second_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operations at Airports
Daily IFR arrivals and departures by airport

EUROCONTROL is a pan-European, civil-military organisation dedicated to supporting European aviation.

## Introduction

### Overview of the Dataset

The dataset contains detailed information about flight operations at various airports around the world. This dataset includes key metrics related to flight departures, arrivals, and total flights, both under Instrument Flight Rules (IFR) and general flight operations.

### Dataset Description

The dataset comprises the following columns:

- **YEAR**: The year in which the flight data was recorded.
- **MONTH_NUM**: The numerical representation of the month (e.g., 1 for January, 2 for February).
- **MONTH_MON**: The abbreviated name of the month (e.g., JAN for January).
- **FLT_DATE**: The date of the flight operations.
- **APT_ICAO**: The ICAO code of the airport.
- **APT_NAME**: The name of the airport.
- **STATE_NAME**: The name of the state or country where the airport is located.
- **FLT_DEP_1**: The number of flight departures.
- **FLT_ARR_1**: The number of flight arrivals.
- **FLT_TOT_1**: The total number of flights (departures + arrivals).
- **FLT_DEP_IFR_2**: The number of IFR flight departures (may contain NULL values).
- **FLT_ARR_IFR_2**: The number of IFR flight arrivals (may contain NULL values).
- **FLT_TOT_IFR_2**: The total number of IFR flights (departures + arrivals, may contain NULL values).

### Purpose of the Dataset

This dataset is intended for use in aviation analysis, airport operations management, and transportation research. It can be utilized to:

- Analyze flight traffic patterns and trends over time.
- Study the distribution of flight operations across different airports and regions.
- Examine the impact of IFR conditions on flight operations.
- Explore seasonal variations in flight activity.

# Install and Import necessary Libraries and Pyspark

In [ ]:
# Install PySpark in Google Colab
!pip install pyspark

: 

In [ ]:
#Import the pySpark
import pyspark

#Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#Creating a Spark Session
spark = SparkSession.builder.appName("Airport_Traffic").getOrCreate()

In [ ]:
#Mounting google drive to access dataset
from google.colab import drive

drive.mount('/content/gdrive')

# Loading the Dataset from Google Drive

In [ ]:
#Loading the dataset
file_path = "/content/gdrive/MyDrive/Colab Notebooks/Second Project/airport_traffic_2024.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first few rows
df.show(5)

# Display the schema of the dataset
df.printSchema()

# Some Exploration

In [ ]:
#number of columns
len(df.columns)

In [ ]:
#return the total number of rows (or records) present in the df DataFrame.
df.count()

In [ ]:
# to count the number of unique rows in a DataFrame called df.
df.distinct().count()

## Selecting Specific Month

In [ ]:
df_december = df.filter(col("MONTH_NUM") == 12)
df_december.show(5)

In [ ]:
#return the total number of rows (or records) present in the df DataFrame.
df_december.count()

In [ ]:
# to count the number of unique rows in a DataFrame called df.
df_december.distinct().count()

In [ ]:
df_december.groupBy("STATE_NAME").count().show()

In [ ]:
#Ordering the number of Airports by State
df_december.groupBy("STATE_NAME").count().orderBy(desc("count")).show()

In [ ]:
#Total number of flights
df_december_summary = df_december.groupBy("STATE_NAME").agg(
    sum("FLT_TOT_1").alias("Total December Flights")
).orderBy(col("Total December Flights").desc()).show()

This shows us busiest state is Spain and France has most airports. However when we check Google, total number of airports in France is 169. That means we have some issues with our calculation.

In [ ]:
#Let's count again. Now, we will use "countDistinct" to calculate the unique airports per state
df_december.groupBy("STATE_NAME").agg(
    countDistinct("APT_ICAO").alias("Unique Airports")
).orderBy(col("Unique Airports").desc()).show()

In [ ]:
df_december_Türkiye = df_december.filter(col("STATE_NAME") == "Türkiye")
df_december_Türkiye.show()

In [ ]:
df_december_Türkiye.groupBy("APT_NAME").agg(
    countDistinct("APT_NAME").alias("Airport Numbers")).show()

In [ ]:
df_december_Türkiye.agg(countDistinct("APT_NAME").alias("Airport Numbers")).show()

In [ ]:
df_december_Türkiye.show()

In [ ]:
df_december_Türkiye.filter(col("FLT_TOT_1") > 100).orderBy(desc("FLT_TOT_1")).show()

## Inspecting Data

In [ ]:
df_december_Türkiye.printSchema()

In [ ]:
df_december_Türkiye.describe().show()

In [ ]:
print(df_december_Türkiye.dtypes)

In [ ]:
print(df_december_Türkiye.columns)

In [ ]:
df_december_Türkiye.count()

In [ ]:
first_three_rows = df_december_Türkiye.take(3)
print(first_three_rows)

In [ ]:
is_empty = df_december_Türkiye.isEmpty()
print(f"Is the DataFrame empty? {is_empty}")

In [ ]:
df_december_Türkiye.cache()

In [ ]:
df_unique_airports = df_december_Türkiye.select("APT_NAME").distinct()
df_unique_airports.show()

In [ ]:
df_unique_airports = df_december_Türkiye.select("FLT_DEP_1").distinct()
df_unique_airports.show()


In [ ]:
sample_data = df_december_Türkiye.sample(fraction=0.1)
sample_data.show()

In [ ]:
sample_data.count()

In [ ]:
first_row = df_december_Türkiye.first()
print(first_row)

In [ ]:
limited_data = df_december_Türkiye.limit(10)
limited_data.show()

# Data Cleaning

## Checking for Null Values

In [ ]:
df_december_Türkiye.filter(col("APT_ICAO").isNull()).show()
df_december_Türkiye.filter(col("APT_NAME").isNull()).show()
df_december_Türkiye.filter(col("STATE_NAME").isNull()).show()
df_december_Türkiye.filter(col("FLT_DEP_1").isNull()).show()
df_december_Türkiye.filter(col("FLT_ARR_1").isNull()).show()
df_december_Türkiye.filter(col("FLT_TOT_1").isNull()).show()

In [ ]:
null_count = df_december_Türkiye.filter(col("APT_ICAO").isNull()).count()
print(f"Number of null values in APT_ICAO: {null_count}")

null_count = df_december_Türkiye.filter(col("APT_NAME").isNull()).count()
print(f"Number of null values in APT_NAME: {null_count}")

null_count = df_december_Türkiye.filter(col("STATE_NAME").isNull()).count()
print(f"Number of null values in STATE_NAME: {null_count}")

null_count = df_december_Türkiye.filter(col("FLT_DEP_1").isNull()).count()
print(f"Number of null values in FLT_DEP_1: {null_count}")

null_count = df_december_Türkiye.filter(col("FLT_ARR_1").isNull()).count()
print(f"Number of null values in FLT_ARR_1: {null_count}")

null_count = df_december_Türkiye.filter(col("FLT_TOT_1").isNull()).count()
print(f"Number of null values in FLT_TOT_1: {null_count}")

In [ ]:
df_december_Türkiye = df_december_Türkiye.dropna(subset=["FLT_TOT_1"])


In [ ]:
df_december_Türkiye = df_december_Türkiye.fillna({"FLT_TOT_1": 0})


In [ ]:
# Filter for rows where FLT_TOT_1 is 0
zero_flights_count = df_december_Türkiye.filter(col("FLT_TOT_1") == 0).count()

# Print the count
print(f"Number of rows with FLT_TOT_1 equal to 0: {zero_flights_count}")

In [ ]:
df_december_Türkiye = df_december_Türkiye.withColumnRenamed("FLT_DEP_1", "DEPARTURES")
df_december_Türkiye = df_december_Türkiye.withColumnRenamed("FLT_ARR_1", "ARRIVALS")
df_december_Türkiye = df_december_Türkiye.withColumnRenamed("FLT_TOT_1", "TOTAL FLIGHTS")
df_december_Türkiye = df_december_Türkiye.withColumnRenamed("FLT_TOT_1", "TOTAL FLIGHTS")
df_december_Türkiye = df_december_Türkiye.withColumnRenamed("STATE_NAME", "STATE NAME")
df_december_Türkiye = df_december_Türkiye.withColumnRenamed("APT_NAME", "AIRPORT NAME")
df_december_Türkiye = df_december_Türkiye.withColumnRenamed("APT_ICAO", "AIRPORT CODE")
df_december_Türkiye = df_december_Türkiye.withColumnRenamed("MONTH_NUM", "MONTH NO")
df_december_Türkiye = df_december_Türkiye.withColumnRenamed("FLT_DATE", "FLIGHT DATE")
df_december_Türkiye = df_december_Türkiye.withColumnRenamed("YEAR_NUM", "YEAR NO")



In [ ]:
df_december_Türkiye.show(5)

In [ ]:
df_december_Türkiye = df_december_Türkiye.drop("FLT_DEP_IFR_2", "FLT_ARR_IFR_2", "FLT_TOT_IFR_2")
df_december_Türkiye.show()

In [ ]:
df_december_Türkiye.groupBy("AIRPORT NAME").agg(
    sum("TOTAL FLIGHTS").alias("Total Number of Flights")
).orderBy(col("Total Number of Flights").desc()).show()

In [ ]:
df_december_Türkiye.groupBy("AIRPORT NAME").agg(
    sum("DEPARTURES").alias("Total Number of Departures")
).orderBy(col("Total Number of Departures").desc()).show()

In [ ]:
df_december_Türkiye.groupBy("AIRPORT NAME").agg(
    sum("ARRIVALS").alias("Total Number of Arrivals")
).orderBy(col("Total Number of Arrivals").desc()).show()

## Adding New Columns

In [ ]:
df_december_Türkiye.show(5)

In [ ]:
df_december_Türkiye.select("FLIGHT DATE").show()

In [ ]:
df_december_Türkiye.select("FLIGHT DATE").distinct().show()

In [ ]:
df_december_Türkiye.select(countDistinct("FLIGHT DATE")).show()

In [ ]:
df_december_Türkiye.groupBy("AIRPORT NAME").agg(
    sum("TOTAL FLIGHTS").alias("Total Number of Flights"),
    sum("DEPARTURES").alias("Total Number of Departures"),
    sum("ARRIVALS").alias("Total Number of Arrivals")
).show()

In [ ]:
df_december_Türkiye.groupBy("FLIGHT DATE").agg(
    sum("TOTAL FLIGHTS").alias("Total Number of Flights"),
    sum("DEPARTURES").alias("Total Number of Departures"),
    sum("ARRIVALS").alias("Total Number of Arrivals")
    ).orderBy(col("Total Number of Flights").desc()).show()

In [ ]:
df_december_Türkiye.groupBy("FLIGHT DATE", "AIRPORT NAME").agg(
    sum("TOTAL FLIGHTS").alias("Total Number of Flights"),
    sum("DEPARTURES").alias("Total Number of Departures"),
    sum("ARRIVALS").alias("Total Number of Arrivals")
).orderBy(col("Total Number of Flights").desc()).show()

In [ ]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy("AIRPORT NAME").orderBy(col("Total Number of Flights").desc())

df_busiest_days = df_december_Türkiye.groupBy("FLIGHT DATE", "AIRPORT NAME").agg(
    sum("TOTAL FLIGHTS").alias("Total Number of Flights")
).withColumn("rank", row_number().over(window_spec)) \
.filter(col("rank") == 1) \
.drop("rank") \
.orderBy(col("Total Number of Flights").desc()) \
.show()

In [ ]:
df_december_Türkiye = df_december_Türkiye.withColumn('Difference Between Arrivals and Departures', col('ARRIVALS') - col('DEPARTURES')).show()